In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
import yolo_utils
from PIL import Image
import cv2
import imageio
%matplotlib inline

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
    """
    box_confidence: 每个格子的每个anchor有物体的概率，（19*19， 5， 1）
    boxes: 每个格子的每个anchor位置参数，（19*19， 5， 4）
    box_class_probs: 每个格子的每个anchor每个类别的概率， （19*19， 5， 80）
    """
    box_scores = box_confidence*box_class_probs  # 每个格子的每个anchor每个类别的得分 (19*19, 5, 80)
    
    box_classes = K.argmax(box_scores, axis = -1)  # 每个格子的每个anchor得分最高的那个类别的索引 （19*19， 5）
    box_class_scores = K.max(box_scores, axis = -1) # 每个格子的每个anchor得分最高的那个类别的得分值(19*19, 5)
    
    filtering_mask = (box_class_scores >= threshold) # (19*19, 5) True or False 决定了哪个格子的哪个anchor该去掉
    
    scores = tf.boolean_mask(box_class_scores, filtering_mask) #  把得分低的去掉 ，记录了留下来的anchor得分 ,shape:(?, )
    print("score.shape = ", scores.shape)
    boxes =  tf.boolean_mask(boxes, filtering_mask)  # 把得分低的去掉， 记录了留下来的anchor的位置，shape:（？，4）
    print("boxes.shape = ", boxes.shape)
    classes = tf.boolean_mask(box_classes, filtering_mask) # 把得分低的去掉， 记录了留下来的anchor最可能是哪个类别（索引）， shape：（？，）
    return scores, boxes, classes
    

In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19,19,5,1], mean=1, stddev=4, seed=1)
    boxes = tf.random_normal([19,19,5,4],  mean=1, stddev=4, seed=1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)

    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

    test_a.close()

In [ ]:
def iou(box1, box2):
    """
    关键是找到重叠部分的坐标，左上角比较坐标，取大值；右下角，比较坐标，取小值
    """
    x_left_up = np.maximum(box1[0], box2[0])
    y_left_up = np.maximum(box1[1], box2[1])
    x_right_down = np.minimum(box1[2], box2[2])
    y_right_down = np.minimum(box1[3], box2[2])
    
    inter_area = (x_right_down - x_left_up) * (y_right_down - y_left_up)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    nuion_area = box1_area + box2_area - inter_area
    
    iou = inter_area / nuion_area
    return iou
    

In [ ]:
box1 = (2,1,4,3)
box2 = (1,2,3,4)

print("iou = " + str(iou(box1, box2)))

**非极大值抑制**

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_theshold = 0.5):
    """
    scores:（None,）,记录了每个框的得分
    boxes:(None, 4),记录了每个框的位置
    classes：(None, ), 记录了每个框的索引（类别）
    max_boxes：预测锚框数量的最大值
    返回：
    scores：（，None）,每个锚框预测的可能值
    boxes：（4， None）， 预测锚框的坐标
    classes:(, None), 每个锚框的预测的分类
    """
    max_boxes_tensor = K.variable(max_boxes, dtype = "int32") 
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_theshold)
    
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)

    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

    test_b.close()

In [ ]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes = 10, score_threshold = 0.6, iou_theshold = 0.5):
    """
    此函数处理经yolo输出的锚框信息，总的来说，删除不必要的框
    """
    # 获取yolo模型的输出
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    # 中心点转换为角
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    boxes = yolo_utils.scale_boxes(boxes, image_shape)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_theshold) # nms
    return scores, boxes, classes  

In [ ]:
with tf.Session() as test_c:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)

    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

    test_c.close()

**加载模型进行测试**

In [ ]:
sess = K.get_session()
# 加载类别名字和5个anchor
class_names = yolo_utils.read_classes("model_data/coco_classes.txt")
print(type(class_names))
anchors = yolo_utils.read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)

In [ ]:
# 加载模型
yolo_model = load_model("model_data/yolov2.h5")
print(type(yolo_model))

In [ ]:
#yolo_model.summary()

In [ ]:
# 将模型输出转换为边界框
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
def predict(sess, image_file, is_show_info = True, is_plot = True):
    # 图像预处理
    image, image_data = yolo_utils.preprocess_image("images/" + image_file, model_image_size = (608, 608))
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict = {yolo_model.input:image_data, K.learning_phase():0})
    if is_show_info:
        print("There are {} boxes in {}".format(len(out_boxes), image_file))
        
    colors = yolo_utils.generate_colors(class_names)
    yolo_utils.draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    image.save(os.path.join("out", image_file), quality = 100)
    if is_plot:
#         output_image = cv2.imread(os.path.join("out", image_file))
#         output_image = cv2.cvtColor(output_image , cv2.COLOR_BGR2RGB)
#         output_image = Image.fromarray(output_image)
#         output_image.show()
        output_image = imageio.imread(os.path.join("out", image_file))
        plt.imshow(output_image)
    return out_scores, out_boxes, out_classes

In [ ]:
out_scores, out_boxes, out_classes = predict(sess, "test.jpg")